# Example Internal Registry Query

<a target="_blank" href="https://colab.research.google.com/github/dannys0n/CS394-MyModulesRepo/blob/main/notebooks/02/Gradio%20Internal%20HTTP%20Resources%20Registry.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://github.com/dannys0n/CS394-MyModulesRepo/tree/main/notebooks/02/Gradio%20Internal%20HTTP%20Resources%20Registry.ipynb">
  <img src="https://img.shields.io/badge/Download_.ipynb-blue" alt="Download .ipynb"/>
</a>

## Install required packages

In [18]:
!uv pip install gradio==5.49.1 openai

Using Python 3.12.12 environment at: /usr
Audited 2 packages in 128ms


## Set the OpenRouter API Key from Colab Secrets + initialize client

In [63]:
from google.colab import userdata
import openai

OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')

# Initialize OpenAI client
client = openai.OpenAI(
    base_url='https://openrouter.ai/api/v1',
    api_key=OPENROUTER_API_KEY,
)

## Define some datastructures and registry for prompting

In [86]:
INTERNAL_DB_HTTP_REGISTRY = {
  "registry.help": "https://registry.help.internal.local.fakelink",

  "dashboard": "https://dashboard.internal.local.fakelink",
  "dashboard.auth": "https://dashboard.internal.local/auth.fakelink",
  "dashboard.payments": "https://dashboard.internal.local/payments.fakelink",

  "metrics": "https://metrics.internal.local.fakelink",
  "metrics.auth.latency": "https://metrics.internal.local/auth/latency.fakelink",
  "metrics.auth.errors": "https://metrics.internal.local/auth/errors.fakelink",
  "metrics.payments.throughput": "https://metrics.internal.local/payments/throughput.fakelink",

  "api": "https://api.internal.local.fakelink",
  "api.auth": "https://api.internal.local/auth.fakelink",
  "api.auth.login": "https://api.internal.local/auth/login.fakelink",
  "api.auth.refresh": "https://api.internal.local/auth/refresh.fakelink",
  "api.payments": "https://api.internal.local/payments.fakelink",
  "api.payments.charge": "https://api.internal.local/payments/charge.fakelink",
  "api.payments.refund": "https://api.internal.local/payments/refund.fakelink",

  "incident": "https://incident.internal.local.fakelink",
  "incident.auth.outage": "https://incident.internal.local/auth-outage.fakelink",
  "incident.payments.timeout": "https://incident.internal.local/payments-timeout.fakelink",
  "incident.database.degraded": "https://incident.internal.local/db-degraded.fakelink",

  "runbook": "https://runbook.internal.local.fakelink",
  "runbook.auth.latency": "https://runbook.internal.local/auth-latency.fakelink",
  "runbook.auth.token_expiry": "https://runbook.internal.local/auth-token-expiry.fakelink",
  "runbook.payments.retry_storm": "https://runbook.internal.local/payments-retry-storm.fakelink",

  "debug": "https://debug.internal.local.fakelink",
  "debug.auth.session": "https://debug.internal.local/auth/session.fakelink",
  "debug.payments.webhook": "https://debug.internal.local/payments/webhook.fakelink"
}


In [87]:
# system prompts to pass
CHAT_SYSTEM_PROMPT = {
    "role": "system",
    "content": "Your sole purpose is to act as an internal HTTP resources registry retrieval assistant. "
               "always start your response with 'USR_REQ:' followed by a concise summary of the user's request, then stop."
               "Do not ask follow-up questions, do not output json. do not provide links."
               "assume a separate structured output model will take over and query database right after your completion based on your output."
}
STRUCTURED_SYSTEM_PROMPT = {
    "role": "system",
    "content": "From your thinking, create a database resource to return to the user. return null values if not applicable. "
               "Take the following previous output and search the following internal HTTP resources registry (internal_registry_name:internal_registry_link) for the best match: "
                + str(INTERNAL_DB_HTTP_REGISTRY)
}

# models for user to pick in UI
AVAILABLE_MODELS = [
    "mistralai/ministral-8b-2512",
    "qwen/qwen3-14b",
    "openai/gpt-5.2-chat",
]

## Define datatypes for structured output

In [88]:
from pydantic import BaseModel
from typing import List

# data structures for an Itinerary
class DB_Details(BaseModel):
    header_name: str
    short_executive_summary: str
    bulleted_short_explanations: List[str]

class BD_Resource(BaseModel):
    internal_registry_name: str
    internal_registry_link: str
    details: DB_Details




## Define functions for export structured output

In [89]:
import json
import tempfile
from pathlib import Path

def export_output(output_json: str) -> str:
    tmp = tempfile.NamedTemporaryFile(
        mode="w",
        suffix=".json",
        delete=False,
        encoding="utf-8"
    )

    data = json.loads(output_json)
    json.dump(data, tmp, indent=2, ensure_ascii=False)
    tmp.close()

    return tmp.name


## Define functions for streaming and structured output

In [ ]:
# stream output (prompts for structured output + export)
def chat_with_streaming(message, history, model_name):
    messages = [CHAT_SYSTEM_PROMPT]

    if history:
        messages.extend(history)

    messages.append({"role": "user", "content": message})

    stream = client.chat.completions.create(
        model=model_name,
        messages=messages,
        stream=True
    )

    response_text = ""

    # streaming back response
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            token = chunk.choices[0].delta.content
            response_text += token
            yield response_text, None, None

    # stream finished
    # generate structured output
    structured = generate_structured_output(response_text, model_name)
    # generate export
    file_path = export_output(structured)

    # final update (chat + JSON + export)
    yield response_text, structured, file_path

# prompt model for structured output
def generate_structured_output(response_text, model_name):
    messages = [
        STRUCTURED_SYSTEM_PROMPT,
        {
            "role": "user",
            "content": response_text
        }
    ]

    response = client.chat.completions.parse(
        model=model_name,
        messages=messages,
        response_format=BD_Resource
    )
    return response.choices[0].message.content


## Start chat interface

UI showing streaming, structured output, and download export

In [ ]:
import gradio as gr

# Create chat interface
with gr.Blocks() as demo:
    gr.Markdown("## Internal HTTP Resources Registry Retrival AI")

    # dropdown to select model
    model_dropdown = gr.Dropdown(
        choices=AVAILABLE_MODELS,
        value=AVAILABLE_MODELS[0],
        label="Model"
    )

    # grouping outputs
    with gr.Row():
          # defining structured output + download
          with gr.Column(scale=2):
              structured_json = gr.JSON(scale=3,label="Structured Output")

              download_file = gr.File(
                  label="Download Resource",
                  file_types=[".json"],
              )
          # overall chat window
          with gr.Column(scale=1):
              chat = gr.ChatInterface(
                  fn=chat_with_streaming, # call function and pass arguments
                  additional_inputs=[model_dropdown],
                  type="messages",
                  additional_outputs=[
                      structured_json,
                      download_file,
                  ]
              )


demo.launch(share=True)



## Use of AI Tools (Academic Disclosure)

AI-assisted tools (ChatGPT-5.2 Auto) were used during the development of this notebook to:
- Explore design options for a Gradio-based chat interface
- Assist in refining Python code
- Debug runtime and UI issues

All generated content was reviewed, modified, and validated by the author.
The author retains responsibility for the correctness and originality of the final submission.